# Building an AI Code Problem Solver with Hyperbrowser

In this cookbook, we'll create an AI-powered code problem solver that can tackle coding challenges from platforms like LeetCode, HackerRank, and others. Using Hyperbrowser's Browser Use feature with `Claude 3.7 Sonnet` and it's superior coding abilities, we'll build an agent that can:

- Read and understand coding problems directly from the source
- Generate solutions in your preferred programming language
- Test solutions when possible using the platform's interface
- Return clean, template-compliant code

Let's turn complex coding challenges into solved problems with just a few lines of code!

## Prerequisites

To follow along, you'll need:

1. A Hyperbrowser API key (sign up at [hyperbrowser.ai](https://hyperbrowser.ai) if you don't have one)
2. Python 3.9+ installed

Store your API key in a `.env` file in the same directory as this notebook:

```shell
HYPERBROWSER_API_KEY=your_hyperbrowser_key_here
```

## Setup

First, let's import our dependencies and initialize the Hyperbrowser client:

In [5]:
import os

from dotenv import load_dotenv
from hyperbrowser import AsyncHyperbrowser
from hyperbrowser.models import (
    StartBrowserUseTaskParams,
    CreateSessionParams,
    ScreenConfig,
)

load_dotenv()

True

In [6]:
hb = AsyncHyperbrowser(api_key=os.getenv("HYPERBROWSER_API_KEY"))

## Creating the Code Solver

Our code solver function combines Browser Use with Claude 3.7 Sonnet to create a powerful coding assistant. The agent will:

1. Visit the provided coding problem URL
2. Analyze the problem requirements and constraints
3. Generate a solution in the specified language
4. Ensure the solution follows any provided templates

The default LLM used by the Browser Use agent is `Gemini 2.0 flash`, which while powerful is not as good as `Claude 3.7 sonnet` at coding. By mixing up which llm we use, we can leverage the specific abilities of different LLMs to our advantage.

In [7]:
async def perform_task(code_problem_link: str):
    resp = await hb.agents.browser_use.start_and_wait(
        StartBrowserUseTaskParams(
            task=f"""You are an expert programmer. You are required to solve the problem on Leetcode. The problem is available at {code_problem_link}. Return to me the solution to the problem in the language specified.
            The steps for this are

             - Navigate to the provided URL
             - Wait for the page to load completely
             - Identify the problem task. 
                - The problem task is usually listed on the left side of the page. 
                - The problem will have a title, giving the general idea of the task
                - There will be a description describing what had to be done
                - A few examples listing what the inputs and outputs to the solution would be like
                - A number of constraints on the solution
             - Now you'll select the language specified
                - Usually this is to the right side of the page. It will have something like C, C++, Java, Python or something similar listed.
                - Click on this
                - Wait for the list of supported languages to pop up
                - Find the options from among the ones listed that matches the requirements
                - Click on the option
             - Find the template for the solution function.

                - The template for the function should be to the right side of the page. 
                - If will describe what the solution should look like and what parameters it should accept
             - Extract the solution template 
             - Write code that satisfies the problem requirement
                - That means that the solution should satisfy the function template
                - It should also satisfy the function description and title. 
                - If the hypothetical function would be provided with the example inputs, it should be able to produce the example outputs.
                - Return the complete code along with fitting it appropriately inside the function template. 
                - Remove any language specific comments. Return the solution without any comments
             
            Solve the code in typescript
            """,
            use_vision=True,
            session_options=CreateSessionParams(
                screen=ScreenConfig(width=1920, height=1080)
            ),
            llm="claude-3-7-sonnet-20250219"
        )
    )
    if resp.data is not None:
        return resp.data.final_result
    return None

## Testing the Solver

Let's test our code solver on a challenging problem - finding the median of two sorted arrays from LeetCode. This is a great test case because it:

- Requires efficient algorithmic thinking
- Has clear input/output requirements
- Tests the agent's ability to handle complex problem constraints

In [8]:
link_to_coding_task = "https://leetcode.com/problems/median-of-two-sorted-arrays"

response = await perform_task(link_to_coding_task)
if response is not None:
    print(response)
else:
    print("No response from the agent")

function findMedianSortedArrays(nums1: number[], nums2: number[]): number {
    if (nums1.length > nums2.length) {
        [nums1, nums2] = [nums2, nums1];
    }
    
    const m = nums1.length;
    const n = nums2.length;
    const totalLength = m + n;
    const halfLength = Math.floor((totalLength + 1) / 2);
    
    let left = 0;
    let right = m;
    
    while (left <= right) {
        const midX = Math.floor((left + right) / 2);
        const midY = halfLength - midX;
        
        const maxX1 = midX === 0 ? Number.NEGATIVE_INFINITY : nums1[midX - 1];
        const minX1 = midX === m ? Number.POSITIVE_INFINITY : nums1[midX];
        
        const maxY1 = midY === 0 ? Number.NEGATIVE_INFINITY : nums2[midY - 1];
        const minY1 = midY === n ? Number.POSITIVE_INFINITY : nums2[midY];
        
        if (maxX1 <= minY1 && maxY1 <= minX1) {
            if (totalLength % 2 === 1) {
                return Math.max(maxX1, maxY1);
            }
            return (Math.max(maxX1,

![Code Solver Example with Browser Use](/code-solver-browser-use.png)

## Make It Your Own

Here are some ways to customize and extend the code solver:

- Modify the language preference in the task prompt
- Add support for specific coding platforms
- Implement solution validation and testing
- Create a solution archive for future reference
- Add explanation generation for complex solutions

## Conclusion

We've built a powerful AI coding assistant that can tackle complex programming challenges with ease. By combining Hyperbrowser's Browser Use with `Claude 3.7 Sonnet`, we've created a tool that not only solves problems but ensures solutions are properly formatted and tested.

This example also shows how we can leverage the individual abilities of specific LLMs for our benefit. For every problem, it might be worth it to change the LLM used before moving to more intricate approaches.

Whether you're practicing for interviews, learning new algorithms, or just want to save time on coding challenges, this agent has you covered.

## Relevant Links
- [Hyperbrowser](https://hyperbrowser.ai)
- [Hyperbrowser Documentation](https://docs.hyperbrowser.ai)
